# MaskLib version

In [22]:
import os, sys
import time
sys.path.append(r'G:\.shortcut-targets-by-id\1NcciYtLAajpr6Vl_5HlosoVGSzqVLRqY\Hung Lab\Group members\Mingi\2_codes\datamgnt')
sys.path.append(r'G:\.shortcut-targets-by-id\1NcciYtLAajpr6Vl_5HlosoVGSzqVLRqY\Hung Lab\Group members\Mingi\2_codes')
sys.path.append(r'C:\Users\20141\OneDrive - purdue.edu\PhD_research\codes\LABcodes')

from Experiment.datamanagement import SlabFile
from Experiment.dataanalysis import get_next_filename, get_current_filename, make_datapath, current_datapath, figure_style
from Experiment.fit_codes import zipsort, argselectdomain, selectdomain, fithanger_new_withQc, fitgeneral, peakdetect,hangerfunc_new_withQc
import json
import matplotlib.pyplot as plt
import re
import h5py
from tqdm import tqdm

import numpy as np
from numpy import cos, sin, tan, arctan, arcsin, arccos, pi
import matplotlib.pyplot as plt

In [23]:
expt_path = os.getcwd() 
print("Current working dir : %s" % expt_path)

if not os.path.exists('data'):
    os.makedirs('data')
data_path = expt_path + '\\data\\'

print(data_path)

Current working dir : G:\.shortcut-targets-by-id\1NcciYtLAajpr6Vl_5HlosoVGSzqVLRqY\Hung Lab\Group members\Mingi\2_codes\grating_coupler
G:\.shortcut-targets-by-id\1NcciYtLAajpr6Vl_5HlosoVGSzqVLRqY\Hung Lab\Group members\Mingi\2_codes\grating_coupler\data\


In [24]:
# -*- coding: utf-8 -*-
"""
Created on 8/9/2025

@author: mingi

Generating file for a wafer of


Inductor_wiggles(chip,structure,length=None,nTurns=None,maxWidth=None,Width=None,CCW=True,start_bend = True,stop_bend=True,pad_to_width=None,w=None,s=None,radius=None,bgcolor=None,**kwargs)
"""
import numpy as np

from dxfwrite import DXFEngine as dxf
from dxfwrite import const
from dxfwrite.vector2d import vadd

import maskLib.MaskLib as m
from maskLib.dcLib import ResistanceBarNegative, ResistanceBar
from maskLib.microwaveLib import Strip_pad, Strip_bend, Strip_straight, Strip_taper, CPW_launcher, CPW_straight, CPW_stub_open, CPW_stub_short, CPW_stub_round, CPW_bend, CPW_wiggles, CPW_taper, TwoPinCPW_straight, TwoPinCPW_wiggles
from maskLib.microwaveLib import waffle, waffle_region, Inductor_wiggles, TwoPinCPW_wiggles, Strip_wiggles, CPW_tee, CPW_cap, Strip_pad, Strip_stub_short, Strip_stub_open, Wire_bend

from maskLib.utilities import doMirrored
from maskLib.markerLib import MarkerSquare, MarkerCross,MarkerRect, AlphaNumStr

from maskLib.resonatorLib import JellyfishResonator
from maskLib.junctionLib import JContact_slot, JContact_tab

In [25]:
from maskLib.MaskLib import HiVisMarker09

?HiVisMarker09

Signature: HiVisMarker09(dwg, xpos, ypos, number, width, bg=None, **kwargs)
Docstring: <no docstring>
File:      c:\users\20141\onedrive - purdue.edu\phd_research\codes\labcodes\masklib\masklib.py
Type:      function

In [5]:
#w = m.Wafer('CPWResExample',data_path,7000,7000,padding=2500,waferDiameter=m.waferDiameters['2in'],sawWidth=200,frame=1,solid=0,multiLayer=1)

w = m.Wafer(name='20250809GratingCouplerTest', path=data_path, chipWidth=12000, chipHeight=12000, waferDiameter=m.waferDiameters['2in'], padding=2500, sawWidth=70, frame=True, markers=True, solid=False, multiLayer=True, singleChipRow=False, singleChipColumn=False)

#set wafer properties
# w.frame: draw frame layer?
# w.solid: draw things solid?
# w.multiLayer: draw in multiple layers?

# I set saw width to a round number (200 um) to make dicing easier

w.SetupLayers([
    ['BASEMETAL',4],
    ['BUSMAIN',3],
    ['XOR',5],
    ['SECONDLAYER',6],
    ['MARKERS',2]
    ])


#initialize the wafer
w.init()

#write the dicing border
w.DicingBorder()

Number of Chips: 4


## Helper functions

In [ ]:
import math as ma
from scipy import integrate
import matplotlib.pyplot as plt
from maskLib.Entities import SolidPline
from dxfwrite.mixins import SubscriptAttributes
from dxfwrite import const
from dxfwrite.algebra import rotate_2d
from dxfwrite.base import DXFList,dxfstr
from dxfwrite.entities import Polyline, Solid
from dxfwrite.vector2d import vadd, vsub
from dxfwrite import const

def alignmarker_gen(marker0, ebeam_writefield, markersize=10, offset_x=0, offset_y=0):
    X = ebeam_writefield/2+offset_x
    Y = ebeam_writefield/2+offset_y
    marker1 = [tuple(map(lambda a,b: a+b, marker0, (X,Y))), 
               tuple(map(lambda a,b: a+b, marker0, (X+markersize, Y+markersize))), 
               tuple(map(lambda a,b: a+b, marker0, (X+markersize/2, Y+markersize/2)))]
    marker2 = [tuple(map(lambda a,b: a+b, marker0, (-X,Y))), 
               tuple(map(lambda a,b: a+b, marker0, (-X-markersize, Y+markersize))),  
               tuple(map(lambda a,b: a+b, marker0, (-X-markersize/2, Y+markersize/2)))]
    marker3 = [tuple(map(lambda a,b: a+b, marker0, (X, -Y))), 
               tuple(map(lambda a,b: a+b, marker0, (X+markersize, -Y-markersize))), 
               tuple(map(lambda a,b: a+b, marker0, (X+markersize/2, -Y-markersize/2)))]
    marker4 = [tuple(map(lambda a,b: a+b, marker0, (-X,-Y))), 
               tuple(map(lambda a,b: a+b, marker0, (-X-markersize, -Y-markersize))), 
               tuple(map(lambda a,b: a+b, marker0, (-X-markersize/2, -Y-markersize/2)))]
    return (marker1, marker2, marker3, marker4)


class Config:
    # Layer configuration
    LAYER_NAMES = ['0', 'add', 'drop', 'res', 'ss1', 'ss2', 'taper', 'ind', 
                'mark', 'label', 'window', 'chip', 'pulley', 'turn', 'holes', 'grating']
    
    # Character conversion table
    CONV_TABLE = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                'A', 'B', 'C', 'D', 'E', 'F')
    P_TABLE = {'0', '3', '7', '8', 'C', 'F'}
    
    # Drawing tolerances
    ARC_TOL = 0.000001
    EULER_TOL = 0.000001
    
    # Device parameters
    DEV_STEP = 0.003
    DEV_NAME = 'resonator_array'
    
    # Chip dimensions
    CHIP_WIDTH = 12000.
    CHIP_HEIGHT = 12000.
    STREET_WIDTH = 40.
    
    # Window dimensions        
    window_x = 0.
    window_y = 0.
    WINDOW_CLEARANCE_X = 500.
    WINDOW_CLEARANCE_Y = 625. + 450/2.
    WINDOW_P_WIDTH = 8000.
    WINDOW_P_HEIGHT = 8000.


# def _create_grating_coupler(self, position, rotation_deg=0, mirror_x=False):
#     """Create a grating coupler at specified position with transformation."""
#     # Load the existing GDS file
#     lib = gd.GdsLibrary(infile="gratingcoupler.gds")
#     gc_cell = lib.cells["GC"]
#     offset_dist = [0,0]
#     position =  [a - b for a, b in zip(position, offset_dist)]
#     # Get all polygons from the grating coupler cell
#     polygons_by_spec = gc_cell.get_polygons(by_spec=True)
    
#     # Create a new cell to hold the transformed grating coupler
#     gc_holder = gd.Cell(f"GC_HOLDER_{position[0]}_{position[1]}")
    
#     # Process each polygon in the grating coupler
#     for (layer, datatype), poly_list in polygons_by_spec.items():
#         for polygon in poly_list:
#             # Apply transformation to each polygon
#             pts_trans = transform_points(polygon, origin=(0, 0), 
#                                        rotation_deg=rotation_deg, 
#                                        mirror_x=mirror_x)
#             # Translate to final position
#             pts_trans += position
#             # Add transformed polygon to holder cell
#             gc_holder.add(gd.Polygon(pts_trans, layer=layer, datatype=datatype))
    
#     # Add the holder cell to our library
#     self.cell_list.append(gc_holder)
#     self.cell_names.append(gc_holder.name)
    
#     # Add the holder cell to the grating layer
#     self.cell_list[self.cell_names.index('grating')].add(gc_holder)
    
#     return gc_holder


class EulerBend:
    """Helper class for Euler bend calculations."""
    @staticmethod
    def alpha(d, p):
        return (2 * np.sqrt(2) * integrate.quad(lambda t: np.sin(t**2), 0, np.sqrt((np.pi * p)/2))[0] + \
            2 / np.sqrt(np.pi * p) * np.sin((np.pi * (1 - p)) / 2))**2 / d**2
    
    @staticmethod
    def sp(d, p):
        return np.sqrt(np.pi * p / EulerBend.alpha(d, p))
    
    @staticmethod
    def rp(d, p):
        return 1 / np.sqrt(np.pi * p * EulerBend.alpha(d, p))
    
    @staticmethod
    def ys(d, p, s):
        return integrate.quad(lambda t: np.cos(EulerBend.alpha(d, p) * t**2 / 2), 0, s)[0]
    
    @staticmethod
    def xs(d, p, s):
        return integrate.quad(lambda t: np.sin(EulerBend.alpha(d, p) * t**2 / 2), 0, s)[0]


class DeviceParameters:
    """Container for device parameters with default values."""
    def __init__(self):
        # Device 0 parameters
        self.start_wg_W_0 = 1.0
        self.shift_wg_W_0 = 0.0
        self.shift_wg_W_k_0 = 0.0
        self.start_ad_wg_W_0 = 0.701
        self.target_rad_0 = 16.025
        self.offset_rad_0 = 0.
        self.start_rad_0 = self.target_rad_0 - self.offset_rad_0
        self.shift_rad_0 = 0.
        self.shift_rad_k_0 = 0.
        self.target_wg_L_0 = 0.3
        self.delta_wg_L_0 = 0.72
        self.offset_wg_L_0 = round(round(self.delta_wg_L_0/2, 3) // Config.DEV_STEP) * Config.DEV_STEP
        self.start_wg_L_0 = round((self.target_wg_L_0 - self.offset_wg_L_0) // Config.DEV_STEP) * Config.DEV_STEP
        self.shift_wg_L_0 = round(round(self.delta_wg_L_0/3, 3) // Config.DEV_STEP) * Config.DEV_STEP
        self.shift_wg_L_k_0 = round(round(self.delta_wg_L_0/3/16, 3) // Config.DEV_STEP) * Config.DEV_STEP
        self.start_gap_0 = 0.201
        self.shift_gap_0 = 0.0
        self.shift_gap_k_0 = 0.0
        self.start_clength_0 = 1.95
        self.shift_clength_0 = 0.0
        self.shift_clength_k_0 = 0.0
        self.start_hradh_0 = 5.0
        self.shift_hradh_0 = 0.0
        self.shift_hradh_k_0 = 0.0
        self.start_hradv_0 = 10.0
        self.shift_hradv_0 = 0.0
        self.shift_hradv_k_0 = 0.0
        self.bend_radius = 40
        self.bend_direction = 'down'
        self.final_length = 0

        # Array parameters
        self.array_ino = 0  # upright=0, invert=1
        self.array_flip = 1  # enable=1, disable=0
        self.shift_resonator_X = self.target_rad_0 * 60 / 16
        self.array_area_W = 100.
        self.start_resonator_clearance = Config.window_x #0.5 * (Config.WINDOW_P_WIDTH - self.array_area_W)
        self.array_resonator_X = ma.floor(self.array_area_W / self.shift_resonator_X)
        
        # Added parameters from resonator array code
        self.dev_H = 150
        self.dev_aH = 140
        self.start_dev_X = -self.array_area_W/2
        self.start_dev_Y = -0.5*Config.WINDOW_P_HEIGHT + 500
        self.shift_dev_Y = self.dev_H
        self.start_dev_aY = self.start_dev_Y
        self.shift_dev_aY = self.dev_aH
        
        # Waveguide parameters
        self.ad_wg_taper_buffer = 0.
        self.ad_wg_rampcle = 40.
        self.ad_wg_clearance = 10.
        self.ad_wg_bezcoe = 0.3
        self.start_ad_neck_L = 75.
        self.start_ad_waist_L = 0.
        self.start_ad_waist_W = 1.0
        self.start_ad_waist_L_top = 100.0
        self.start_ad_taper_W = 1.0
        self.ad_wg_start = -100    
        
        # Drawing parameters
        self.flin_pts = 800
        self.plin1_pts = 785
        self.plin2_pts = 30

#def Ring(chip,structure,params=DeviceParameters(),bgcolor=None,**kwargs):
class Ring(SolidPline):
    '''
    Ring Resonator with Euler band
    '''

    name="EULERRING"

    def __init__(self,insert, params=DeviceParameters(),parity=0, rotation=0.,color=const.BYLAYER,bgcolor=None,layer='0',linetype=None,ralign=const.BOTTOM,valign=const.BOTTOM,vflip=False,hflip=False, w=1, **kwargs):

        self.insert = insert
        self.rotation = ma.radians(rotation)
        self.color = color
        self.bgcolor = bgcolor
        self.layer = layer
        self.linetype = linetype
        self.width = w
        
        self.points = []
        self.valign = valign
        self.ralign = ralign
        self.parity = parity
        
        self.vflip = vflip and -1 or 1
        self.hflip = hflip and -1 or 1
        
        
        # Initialize counters
        self.text_ix = 0
        self.gov_ix = 0
        self.ar_ix = 0
        self.tap_ix = 0
        self.res_ix = 0
        self.res_ix_0 = 0
        

        self.params = params

        # Initialize device parameters
        self.devices = [0]
        self.dev_num = len(self.devices)    
        self.fc_num_tot = 30
        self.ar_row_tot = 16
        self.ar_row_h = self.ar_row_tot / 2
        self.fc_cur = 0
        
        # Initialize counters
        self.text_ix = 0
        self.gov_ix = 0
        self.ar_ix = 0
        self.tap_ix = 0
        self.res_ix = 0
        self.res_ix_0 = 0

        # Initialize positions
        dev_X = self.params.start_dev_X
        dev_Y = self.params.start_dev_Y + self.params.shift_dev_Y * self.gov_ix
        dev_aY = self.params.start_dev_aY + self.params.shift_dev_aY * self.ar_ix - dev_Y
        start_wg_W_ = self.params.start_ad_waist_W
        ad_wg_W = self.params.start_ad_wg_W_0
        
        # Array settings (exact from reference)
        sign_k_ini = self.params.array_ino  # upright=0, invert=1
        flip_array = self.params.array_flip  # enable=1, disable=04

    def _get_radius_align(self):

        #by default the radius is defined as the inner radius
        if self.ralign == const.MIDDLE:
            dr = -self.height/2.
        elif self.ralign == const.TOP:
            dr = -self.height
        else:  # const.BOTTOM
            dr = 0.

        return (0, dr)

    def _get_align_vector(self):

        #note: vertical alignment is flipped from regular rectangle
        if self.valign == const.MIDDLE:
            dy = -self.height/2.
        elif self.valign == const.TOP:
            dy = -self.height
        else:  # const.BOTTOM
            dy = 0.

        return (0, dy)

    def _build(self):
        data = DXFList()
        ralign = self._get_radius_align()
        self.points = self._calc_points(self.parity)
        
        align_vector = list(self._get_align_vector())
        offset_x = 0.5*(np.max(self.points[:,0]) - np.min(self.points[:,0]))
        offset_y = 0.5*(np.max(self.points[:,1]) + np.min(self.points[:,1]))
        print(align_vector)
        print(f"offset x ={offset_x}")
        print(f"offset y = {offset_y}")
        # update align vector with ring geometry offset
        align_vector[0] -= offset_x
        align_vector[1] -= offset_y
        
        self._transform_points(align_vector)
        if self.color is not None:
            data.append(self._build_polyline())
        if self.bgcolor is not None:
            #if _calc_points has been run, rmin is already set
            if self.rmin <= 0:
                #if self.angle%(2*math.pi) == math.radians(90): #rounded corner case
                for i in range(self.segments+1):
                    data.append(self._build_solid_triangle(i))
            else: #rmin>0, normal operation
                for i in range(self.segments+1):
                    data.append(self._build_solid_quad(i))
        return data
        
    def _transform_points(self,align):
        self.points = [vadd(self.insert,  # move to insert point
                            rotate_2d(  # rotate at origin
                                ((point[0]+align[0])*self.hflip,(point[1]+align[1])*self.vflip), self.rotation))
                       for point in self.points]

    def _calc_points(self, parity):
        k = parity
        #dev_Y_shift = k // self.params.array_resonator_X * self.params.shift_dev_Y
        #dev_aY_shift = k // self.params.array_resonator_X * self.params.shift_dev_aY
        
        # Calculate resonator parameters (exact from reference)
        res_W = (self.params.start_wg_W_0 + self.params.shift_wg_W_0 * self.res_ix_0 + self.params.shift_wg_W_k_0 * k)
        
        res_length = (self.params.start_wg_L_0 + self.params.shift_wg_L_0 * self.res_ix_0 + self.params.shift_wg_L_k_0 * k)
        
        rad = (self.params.start_rad_0 + self.params.shift_rad_0 * self.res_ix_0 + self.params.shift_rad_k_0 * k)
        
        hradh = (self.params.start_hradh_0 + self.params.shift_hradh_0 * self.res_ix_0 + self.params.shift_hradh_k_0 * k)
        
        hradv = (self.params.start_hradv_0 + self.params.shift_hradv_0 * self.res_ix_0 + self.params.shift_hradv_k_0 * k)
        
        gap = (self.params.start_gap_0 + self.params.shift_gap_0 * self.res_ix_0 + self.params.shift_gap_k_0 * k)
        
        clength = (self.params.start_clength_0 + self.params.shift_clength_0 * self.res_ix_0 + self.params.shift_clength_k_0 * k)
        
        # Pulley calculations (exact from reference)
        pulley_rad = rad + 0.5 * (gap + res_W)
        #pulley_wg_rad = rad + gap + 0.5 * (ad_wg_W + res_W)
        pulley_theda = clength / pulley_rad
        #pulley_brad = (0.5 * self.params.shift_resonator_X - pulley_wg_rad * np.sin(0.5 * pulley_theda)) / np.sin(0.5 * pulley_theda)
        #pulley_shift_Y = (pulley_wg_rad + pulley_brad) * (1 - np.cos(0.5 * pulley_theda))
        
        # Create resonator path (exact from reference)
        #res_spec = set_layer_spec(self.config.LAYER_NAMES, self.spec, 'res')
        dia = rad * 2.
        ep = 1 - pulley_theda / np.pi
        
        flin = np.linspace(0, EulerBend.sp(dia, 1), self.params.flin_pts)
        fex = np.array([EulerBend.xs(dia, 1, i) for i in flin])
        fey = np.array([EulerBend.ys(dia, 1, i) for i in flin])
        
        plin1 = np.linspace(0, EulerBend.sp(dia, ep), self.params.plin1_pts)
        plin2 = np.linspace(np.pi/2 * ep, np.pi/2, self.params.plin2_pts)
        
        
        pex1 = np.array([EulerBend.xs(dia, ep, i) for i in plin1])
        pey1 = np.array([EulerBend.ys(dia, ep, i) for i in plin1])
        pex2 = np.array([dia/2 - EulerBend.rp(dia, ep) * np.sin(np.pi/2 - i) for i in plin2])
        pey2 = np.array([EulerBend.ys(dia, ep, EulerBend.sp(dia, ep)) - 
                    EulerBend.rp(dia, ep) * np.cos(np.pi/2 * (1 - ep)) + 
                    EulerBend.rp(dia, ep) * np.cos(np.pi/2 - i) for i in plin2])
        
        # self.offsetex = np.max(pex2)
        # self.offsetey = np.max(pey2)
        
        pexline = np.concatenate(([fex[0]], 
                                  fex, 
                                  dia - np.flip(fex)[1:], 
                                  dia - pex1[1:], 
                                  dia - pex2[1:], 
                                  np.flip(pex2)[1:], 
                                  np.flip(pex1)[1:]))
        
        peyline = np.concatenate(([fey[0]], 
                                   fey + res_length, 
                                   np.flip(fey)[1:] + res_length, 
                                  -pey1[1:], 
                                  -pey2[1:], 
                                   np.flip(-pey2)[1:], 
                                   np.flip(-pey1)[1:]))
        
        peline = np.column_stack((pexline, peyline))
        return peline

    def _build_polyline(self):
        '''Build the polyline (key component)'''
        polyline = Polyline(self.points, startwidth=self.width, endwidth=self.width ,layer=self.layer,color=self.color, flags=0)
        #polyline.close() #redundant in most cases
        if self.linetype is not None:
            polyline['linetype'] = self.linetype
        return polyline
    
    def _build_solid_quad(self,i,center=None):
        ''' build a single background solid quadrangle segment '''
        solidpts = [self.points[j] for j in [i,i+1,-i-2,-i-1]]
        return Solid(solidpts, color=self.bgcolor, layer=self.layer)  

    
    def _build_solid_triangle(self,i):
        ''' build a single background solid quadrangle segment '''
        solidpts = [self.points[j] for j in [0,i+1,i+2]]
        return Solid(solidpts, color=self.bgcolor, layer=self.layer)  
    
    def __dxf__(self):
        ''' get the dxf string '''
        return dxfstr(self.__dxftags__())
    
    def __dxftags__(self):
        return self._build() 



## wafer define

In [ ]:
A_b = 100
B_b = 300

class GCRingTest(m.Chip12mm):
    def __init__(self,wafer,chipID,layer,
                 total_lengths = [1000, 2000, 3000, 4000],#total cpw length (sets the resonator frequency) (lo to high freq)
                 seps =       [10]*2,# resonator distance to cpw (sets each resonator's coupling)
                 indices =    [0,2,1,3], #these indices are chosen so no two adjacent resonators are close in frequency (to limit crosstalk)
                 res_spacing=500 # how far apart the resonators are
                 ):
        m.Chip12mm.__init__(self,wafer,chipID,layer,defaults={'w':1, 's':1, 'radius':300,'r_out':10,'r_ins':10,'curve_pts':30})
        #width = 10um, spacing = 6um
        
        for s in self.structures:
            #move away from edge of chip
            #s.shiftPos(340)
            s.shiftPos(distance=150, angle=0, newDir=None)
            
        #optical markers
        #doMirrored(MarkerCross, self, (2900,2900),linewidth=5, chipCentered=True,layer='MARKERS')
        '''
        note that the markercross function is
        '''
        #  MarkerCross(w,pos,size=(200,200),linewidth=80,bgcolor=None,layer=None,chipCentered=False,**kwargs)
        doMirrored(func=MarkerCross, canvas=self, pos=(4000, 4000), linewidth=5, chipCentered=True,layer='MARKERS')

        half_trace = self.defaults['w']/2 + self.defaults['s']

        Strip_pad(chip=self,structure=0,length=100,r_out=2, w=10,bgcolor=None,layer='BUSMAIN')

        DC1 = m.Structure(self,self.centered((0,0)), direction=0)
        #DC5 = self.structures[2]

        self.add(Ring(DC1.start, DeviceParameters(), color=const.BYLAYER,bgcolor=None,layer='BUSMAIN',linetype=None,ralign=const.BOTTOM,valign=const.BOTTOM,vflip=False,hflip=False, w=1))
        DC11 = DC1.cloneAlongLast((500, 0),angle=0, newDirection=0)
        
        self.add(Ring(DC11.start, DeviceParameters(), color=const.BYLAYER,bgcolor=None,layer='BUSMAIN',linetype=None,ralign=const.BOTTOM,valign=const.BOTTOM,vflip=False,hflip=False, w=1))
        
        
        #calculate separation between the two structures
        #xdist = self.structures[5].start[0] - self.structures[0].start[0]
        xdist = 100
        ydist = self.structures[9].start[1] - self.structures[1].start[1]
        
        feedline = self.structures[1]

        s0 = feedline.cloneAlongLast()
     
        
      
#####################################################################################################################

GTChip = GCRingTest(w,'RESONATORS','BASEMETAL')
GTChip.save(w,drawCopyDXF=True,dicingBorder=False,center=True)



[0, 0.0]
offset x =16.025
offset y = -0.022681228439063617
[0, 0.0]
offset x =16.025
offset y = -0.022681228439063617
Saved as: G:\.shortcut-targets-by-id\1NcciYtLAajpr6Vl_5HlosoVGSzqVLRqY\Hung Lab\Group members\Mingi\2_codes\grating_coupler\data\20250809GratingCouplerTest_CHIP_RESONATORS.dxf
